# Kernels

In [14]:
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, fbeta_score, accuracy_score


from scripts.nested_CV import nested_cv
from scripts.skwrapped_kernals import dtw_SVC, rbf_SVC, poly_SVC

In [15]:
# data
file_path = "../data/laser.mat"
mat = loadmat(file_path)

X = mat["X"]
y = mat["Y"].reshape(200)

print(X.shape, y.shape)

(200, 60) (200,)


In [16]:
# configs
seed = 571489  # for repeatability
np.random.seed(42)

fbeta_scorer = make_scorer(fbeta_score, beta=2, pos_label=-1)


In [17]:
# normalizing the data
X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

# Support Vector Machines

Classifiers, that use the hinge loss and use the L2 regularizer

In the dual mode, they can be used in conjunction with kernel methods, which is what is employed in the following notebook.


## 1. SVC with Polynomial Kernel

### Description
The idea here is to map the data into a higher dimensional space, such that data is linearly seperable in that space and we can then perform linear classification

The polynomial kernel is:
$$ k_{\text{poly}}(x, x') = (x^{\text{T}}x' + c)^{p} 

### Nested CV
We run a nested cross validation function for the following hyperparameters of the Polynomial Kernel:
- p - degree of polynomial
- lambda (C) - regularization coefficient

In [18]:
param_grid_poly = {
    'C' : [.001,.01,.1,1,10,100],
    'degree' : [2,3,4,5,6]
}

results_poly = nested_cv(poly_SVC(), X, y, param_grid_poly, scoring=fbeta_scorer, random_state=seed)
print(">>>>RESULTS<<<<")
print("Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV ", results_poly["mean_score"])
print("Best parameters ", results_poly["star_params"])

>>>>RESULTS<<<<
Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV  0.9740150485636798
Best parameters  {'C': 0.001, 'degree': 4}


In [19]:
print(results_poly)

{'mean_score': 0.9740150485636798, 'star_params': {'C': 0.001, 'degree': 4}, 'outer_scores': [0.9803921568627451, 0.9595959595959596, 0.9595959595959596, 0.9803921568627451, 0.9900990099009901], 'std_score': 0.012295113550790568, 'best_params': [{'C': 0.001, 'degree': 5}, {'C': 0.001, 'degree': 6}, {'C': 0.001, 'degree': 5}, {'C': 0.001, 'degree': 4}, {'C': 0.001, 'degree': 4}]}


In [20]:
clf_poly = poly_SVC(C=.001, degree=4)
clf_poly.fit(np.delete(X, [35, 162], axis=0), np.delete(y, [35,162], axis=0))

odd_laser_faulty = clf_poly.predict([X[35]])
odd_laser_non_faulty = clf_poly.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[-1] [1]


## 2. SVC with RBF

## Description

The idea here is to use the squard euclidean distance between vectors as a similarity measure. This creates, gaussian "hills" around samples in the original feature space. Furthermore, since we have a dimension for every data point, this kernel maps to an infinite dimensional space.

The RBF Kernel is:
$$ k_{\text{RBF}}(x,x') = exp(-\gamma ||x-x'||^{2})

### Nested CV
We run a nested cross validation function for the following hyperparameters of the RBF Kernel:
- gamma - scaling of the RBF function
- lambda - regularization coefficient


In [27]:
param_grid_rbf = {
    'C' : [.001,.01,.1,1,10,100],
    'gamma' : [.001,.01, .1 , 1,10,100],
}
results_rbf = nested_cv(rbf_SVC(), X, y, param_grid=param_grid_rbf, scoring=fbeta_scorer, random_state=seed)

print(">>>>RESULTS<<<<")
print("Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV ", results_rbf["mean_score"])
print("Best parameters ", results_rbf["star_params"])

>>>>RESULTS<<<<
Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV  0.9540069130692537
Best parameters  {'C': 1, 'gamma': 0.1}


In [28]:
print(results_rbf)

{'mean_score': 0.9540069130692537, 'star_params': {'C': 1, 'gamma': 0.1}, 'outer_scores': [0.9523809523809523, 0.9900990099009901, 0.8762886597938144, 0.9803921568627451, 0.970873786407767], 'std_score': 0.04080706458523105, 'best_params': [{'C': 10, 'gamma': 0.1}, {'C': 1, 'gamma': 0.1}, {'C': 0.001, 'gamma': 10}, {'C': 1, 'gamma': 0.01}, {'C': 10, 'gamma': 0.1}]}


In [29]:
clf_rbf = rbf_SVC(C=.001, gamma=10)
clf_rbf.fit(np.delete(X, [35, 162], axis=0), np.delete(y, [35,162], axis=0))

odd_laser_faulty = clf_rbf.predict([X[35]])
odd_laser_non_faulty = clf_rbf.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[1] [1]


## 3. SVC with DTW

## Description
The idea here is to find the similarity of time series, that may not be aligned/ are out of phase or are moving at different rates and/or are of different lengths. The kernel is in the form of an RBF kernel, but uses the DTW distance of the time series.

Given a metric $d: X \times X \rightarrow \mathbb{R}_{\geq 0}$ on the input space $X$, the family of *DTW Kernels* is given as:

$$ k_{\text{DTW}}(x, x') = e^{- \lambda d_{\text{DTW}}(x, x'; d)}, $$

for sequences $x, x' \in X^+ := \bigcup_{n \geq 1}{X^n}$ of lengths $|x|$ and $|x'|$. The *DTW distance metric* $d_{\text{DTW}}$ is then given by $\gamma(|x|, |x'|)$, where the helper function $\gamma$ is defined recursively via:

$$ \gamma(i, j) = \begin{cases} d(x_i, x_j') + \min\left(\gamma(i-1, j-1), \gamma(i-1, j), \gamma(i, j-1)\right) & (1 \leq i \leq |x|, \, 1 \leq j \leq |x'|), \\ 
\infty & i = 0 \vee j = 0, \\
0 & (i, j) = (0, 0). \end{cases}
$$
Source: Lab jupyter notebooks

A more visual explanation (source: [Herman Kamper](https://www.youtube.com/watch?v=9GdbMc4CEhE)): <br>
![image](../diagrams/DTW.jpg)


### Nested CV
We run a nested cross validation function for the following hyperparameters of the DTW Kernel:
- gamma - smoothing of the DTW function
- d - distance
- lambda - regularization coefficient

In [24]:
param_grid_dtw = {
    'C': [.001, .01, .1, 1, 10, 100],
    'gamma':[.001, .01, .1, 1],
    'inner_dist': ['euclidean', 'squared euclidean']
}

results_dtw = nested_cv(dtw_SVC(), X, y, param_grid_dtw, scoring=fbeta_scorer, random_state=seed)

print(">>>>RESULTS<<<<")
print("Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV ", results_dtw["mean_score"])
print("Best parameters ", results_dtw["star_params"])

>>>>RESULTS<<<<
Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV  0.998019801980198
Best parameters  {'C': 1, 'gamma': 0.1, 'inner_dist': 'euclidean'}


In [25]:
print(results_dtw)

{'mean_score': 0.998019801980198, 'star_params': {'C': 1, 'gamma': 0.1, 'inner_dist': 'euclidean'}, 'outer_scores': [0.9900990099009901, 1.0, 1.0, 1.0, 1.0], 'std_score': 0.003960396039603964, 'best_params': [{'C': 1, 'gamma': 0.1, 'inner_dist': 'euclidean'}, {'C': 1, 'gamma': 0.1, 'inner_dist': 'euclidean'}, {'C': 1, 'gamma': 0.1, 'inner_dist': 'euclidean'}, {'C': 1, 'gamma': 0.1, 'inner_dist': 'euclidean'}, {'C': 1, 'gamma': 0.1, 'inner_dist': 'euclidean'}]}


In [26]:
clf_dtw = dtw_SVC(C=1, gamma=.1)
clf_dtw.fit(np.delete(X, [35, 162], axis=0), np.delete(y, [35,162], axis=0))

odd_laser_faulty = clf_dtw.predict([X[35]])
odd_laser_non_faulty = clf_dtw.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[-1] [-1]
